In [1]:
import json
import glob
import matplotlib.pyplot as plt
import numpy as np
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api.formatters import JSONFormatter
from tqdm.auto import tqdm
import time
import subprocess
import os
from os import listdir
from os.path import isfile, join
import time
import torchaudio
from datasets import Dataset, Audio, load_dataset, load_from_disk

In [2]:
with open('kannada_asr_data_with_transcript.json' , 'r') as kadt:
    kannada_data_unfiltered = json.load(kadt)
    
kannada_data = list()
#we do this so as to remove those without any transcript
for i in kannada_data_unfiltered:
    if i['transcript']==None:
        pass
    else:
        kannada_data.append(i)

In [3]:
len(kannada_data)

876

In [4]:
def process_transcript(transcript):
    transcript_filtered = [x for x in transcript if (x['text'] != '[ಸಂಗೀತ]') and (x['text'] != '[ಪ್ರಶಂಸೆ]')]
    
    segments = []
    segment_sentences = []
    
    min_length = 20
    max_length = 30
    ignore_raw_segment_length = 15
    min_sent_chars = 30
    
    start = transcript_filtered[0]['start']
    sent = ""
    
    for idx in range(len(transcript_filtered) - 1):
        current_raw_segment = transcript_filtered[idx]
        next_raw_segment = transcript_filtered[idx + 1]
        
        sent = sent + " " + current_raw_segment['text']
    
        current_raw_segment_length = next_raw_segment['start'] - current_raw_segment['start']
        end_proposal = next_raw_segment['start']    
        if current_raw_segment_length >= ignore_raw_segment_length:
            end_proposal = current_raw_segment['start'] + current_raw_segment['duration']
        
        if (end_proposal - start >= min_length):
            end = end_proposal

            # add to the segments only if there is no noise
            if len(sent) > min_sent_chars and (end - start < max_length):
                segments.append([start, end])
                segment_sentences.append(sent)

            sent = ""
            start = end
    
    segments = np.array(segments)

    return segments, segment_sentences

In [9]:
mypath = "/home/phoenix/llama/llama.lisp/src/AI/stt/audio_dataset_torch_audio"
mypath_text = "/home/phoenix/llama/llama.lisp/src/AI/stt/text_dataset_torch_text"
audio_metadata =[]
text_metadata = []

uncut_audio_dir = "/home/phoenix/llama/llama.lisp/src/AI/stt/downloads"
uncut_audio_file_list = [f for f in glob.glob(os.path.join(uncut_audio_dir, "*.mp3"))]

#check if the directory to store .txt and coressponding .mp3 is present after cutting the 
if not os.path.isdir(mypath):
    os.makedirs(mypath)
    
if not os.path.isdir(mypath_text):
    os.makedirs(mypath_text)

#iterate through all the video_id names foe which there was transcript
for kannada_individual_json in tqdm(kannada_data):
    transcript = kannada_individual_json['transcript']
    video_id = kannada_individual_json['video'].split("=")[-1]
    segments, segment_sentences = process_transcript(transcript)
    segment_lengths = segments[:, 1] - segments[:, 0]

    #below is w.r.t to text 
    for index,content in enumerate(segment_sentences):
        filename = os.path.join(mypath_text,f'{video_id}_{index}.txt')
        with open(filename , 'w') as segmnt_sentn:
            segmnt_sentn.write(content)

    text_metadata.append([f'{video_id}',len(segment_sentences)])
    with open("/home/phoenix/llama/llama.lisp/src/AI/stt/audio_dataset_torch_audio/audio_metadata.txt","w") as f:
       json.dump(text_metadata,f)

       #below is w.r.t audio 
    arr, sample_rate = torchaudio.load(f'downloads/{video_id}.mp3')
    num_secs = arr.shape[1] / sample_rate
    for index,timestamps in enumerate(segments): 
        t1 = int(timestamps[0]*sample_rate)
        t2 = int(timestamps[1]*sample_rate)
        data_timestamp_audio = arr[:,t1:t2+1]
        torchaudio.save(f'{mypath}/{video_id}_{index}.mp3', data_timestamp_audio,48000)
    
        
    audio_metadata.append([f'{video_id}',len(segments)])
    with open("/home/phoenix/llama/llama.lisp/src/AI/stt/text_dataset_torch_text/text_metadata.txt","w") as f:
        json.dump(audio_metadata, f)
        

  0%|          | 0/3 [00:00<?, ?it/s]

TypeError: string indices must be integers

In [ ]:
audio_metadata

In [ ]:
mypath = "/home/phoenix/llama/llama.lisp/src/AI/stt/audio_dataset_torch_audio"
mypath_text = "/home/phoenix/llama/llama.lisp/src/AI/stt/text_dataset_torch_text"
audio_files = glob.glob(os.path.join(mypath, "*.mp3"))
audio_prompt_files = glob.glob(os.path.join(mypath_text , "*.txt"))

audio_prompt = list()
for file_name in audio_prompt_files:
    with open(file_name,"r") as f:
        a = f.read()
        print(a)
        audio_prompt.append(a)
    print(audio_prompt)
    
ds = (
    Dataset.from_dict(
        {
            "audio": audio_files,
            "prompt": audio_prompt,
            }
        )
        .cast_column("audio", Audio())
        .train_test_split(seed=0)
    )
ds.save_to_disk("stt_ka_dataset_kn-IN_kannada")
ds.push_to_hub("adithyal1998Bhat/stt_synthetic_kn-IN_kannada")
HF_dataset_name = "adithyal1998Bhat/stt_synthetic_kn-IN_kannada"

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



12028752
469321910
